In [155]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

##### Example 1: Autoencoder with Linear layer

In [156]:
transform = transforms.ToTensor()

In [52]:
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5), (0.5))
# ])

In [53]:
mnist_data = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform)

In [54]:
data_loader = DataLoader(dataset=mnist_data,
                         batch_size=64,
                         shuffle=True)

In [55]:
dataiter = iter(data_loader)

In [56]:
images, labels = dataiter.next()

In [99]:
images[0].shape

torch.Size([1, 28, 28])

Write an Autoencoder from scratch.

Autoencoder:
- 1st layer
    + reduce the size of original image to 128 pixels
    + follows by ReLU
- 2nd layer
    + reduce the size to 64 pixels
    + follows by ReLU
- 3rd layer
    + reduce the size to 12 pixels
    + follows by ReLU
- 4th layer: reduce the size to 3 pixels

The decoder do the opposite

In [150]:
import torch
import torch.nn as nn

In [151]:
images.shape, torch.mean(images), torch.max(images), test_im.shape

(torch.Size([64, 1, 28, 28]), tensor(0.1284), tensor(1.), torch.Size([1, 784]))

Write an Autoencoder for `images` from scratch

In [152]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            # the original image's value
            # is between 0 and 1
            # => sigmoid to scale
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [153]:
model = AutoEncoder()

In [154]:
model(test_im).shape

torch.Size([1, 784])

In [102]:
model = AutoEncoder()
loss_func = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [103]:
num_epochs = 3

In [70]:
outputs = []

In [75]:
for epoch in range(num_epochs):
    for (img, _) in data_loader:
        img = img.reshape(-1, 28*28)
        pred = model(img)
        loss = loss_func(pred, img)
        
        loss.backward()
        optim.step()
        optim.zero_grad()
    
    print(f'Epoch {epoch+1}, loss={loss.item():.4f}')
    outputs.append((epoch, img, pred))

Epoch 1, loss=0.0478
Epoch 2, loss=0.0453
Epoch 3, loss=0.0448


In [96]:
transforms.ToPILImage()(outputs[2][1][3].reshape(-1, 28, 28))

In [87]:
outputs[2][2].shape

torch.Size([32, 784])

https://youtu.be/zp8clK9yCro?t=1173